#### VAR Automation

#### Description:
###### This file contains the code for VAR automation, along with csv's for intermediate results such as portfolio holdings, price changes for BOTH AAVE AND COMP

###### Author: Nishchal Gaba
###### Created on: January 13, 2022

In [1]:
# Import packages
import os
import time
import sys
import random
from datetime import datetime, timedelta

import json
import requests
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import os
import json
import boto3
import requests
import numpy as np
import pandas as pd
from io import StringIO
from datetime import timedelta
from datetime import datetime
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport


import matplotlib.pyplot as plt

In [3]:
# Coingecko data fetcher
def getCoinGeckoHistoricalData(_token):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _token(str): Token Symbol (actual token will be fetched from token_names dictionary)
            
        Returns::
            _resDf(pandas dataframe): Returns the dates (yyyy-mm-dd) and prices (USD) in a dataframe
    '''
    try:
        # Making the coingecko historical API request
        coinGeckoTokenCall = token_names[_token]
        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(coinGeckoTokenCall)+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        
        # Converting the result to JSON
        resJSON = request.json()
        
        # Creating empty dataframe
        resDf = pd.DataFrame(resJSON['prices'], columns = ["timestamp", "price"])
        
        # We divide by 1000 to convert the default milliseconds from CoinGecko to seconds
        # Get the human readable date from timestamp
        resDf['timestamp'] = resDf['timestamp'].apply(lambda x: str(datetime.fromtimestamp(x/1000).date()))
        
        # Return the resultant dataframe
        return resDf
        
        
    except Exception as e:
        print("Exception occured while fetching Coingecko Data")
        return str(e)
    

In [4]:
# AAVE V2 data for a particular date
def aaveV2HistoricalData(_date):
    '''
        Fetches the histroical data from Coingecko for a particular token
        
        
        Args::
            _date(str): date for which data is needed in (%m-%d-%Y) format
            
        Returns::
            _resDf(pandas dataframe): Token, CurrentPrice, availableLiquidity, decimals,
                                        totalLiquidity, utilizationRate, Assets
    '''
    
    try:
        # Starting Index
        ind = 0
        
        # Calling the Aave V2
        aaveV2res = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
        resDf = pd.DataFrame(columns=['Token', 'CurrentPrice', 'availableLiquidity','decimals', 'totalLiquidity','utilizationRate','Assets'])
        
        # Iterating through tokens in AaveV2 API
        for token in aaveV2res:
            try:
                resDf.loc[ind] = [token['symbol']] + [float(token['referenceItem']['priceInUsd'])] + \
                                [float(token['availableLiquidity'])] + [float(token['decimals'])]+ \
                                [float(token['totalLiquidity'])]+ [float(token['referenceItem']['utilizationRate'])] + \
                                [float(token['totalDebt'])]

                ind += 1

            except Exception as e:
                print("**", token, e) #str(asset), str(price_mean), str(price_std), str(cap_mean),str(cap_std), str(vol_mean) , str(vol_std))  
    #             errors.append(token)
                # transactionResponse["error"] = True
                print("Exception occured while fetching Aave V2 Token wise Data")
                return str(e)


        # totalLiquidity == Market
        resDf['totalLiquidity']       = resDf['totalLiquidity'] * resDf['CurrentPrice']
        resDf['Assets']               = resDf['Assets'] * resDf['CurrentPrice']
        resDf['Position']             = resDf['Assets'] - resDf['totalLiquidity']
        resDf['Position($,Billion)']  = resDf['Position'] / pow(10, 9)


        # Return the resultant dataframe
        return resDf
        
    except Exception as e:
        print("Exception occured while fetching Aave V2 API Data")
        return str(e)


In [5]:
# def change():
#     query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

#     V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)
#     V2_client                 = Client(transport=V2_sample_transport)
#     response                  = V2_client.execute(gql(query))

#     # Coingecko Mapping for each token name to fetch historical prices
#     names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         "REN"   : "ren",
#         "RENFIL": "renfil",
#         "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }
    
    
#     columns = []
#     columns.append('timestamp')
#     for key in list(names.keys()):
#         columns.append(key+'_price')
#         columns.append(key+'_%change(10)')


#     df        = pd.DataFrame(columns = columns)
#     LOOK_BACK = 375
#     LIMIT     = 5
#     WBTC      = []

#     request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#     data      = request.json()
#     WBTC      = data['prices']
#     tt        = []
#     P         = []
#     for day in data['prices'][-LOOK_BACK:]:
#         tt.append(day[0]/1000)
#         P.append(day[1])
#     df['WBTC_price'] = P
#     df['timestamp']  = tt

#     track = {}
#     for asset in list(names.keys()):#[:LIMIT]:
#         if asset == 'WBTC':
#             continue
#         request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
#         data = request.json()
#         tt = []
#         P = []
        
#         if len(data['prices']) < LOOK_BACK:
#             req = LOOK_BACK - len(data['prices'])
#             addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
#             data['prices'] = addition + data['prices']
#             track[asset] = req
#         else:
#             data['prices'] = data['prices'][-LOOK_BACK:]
        
#         for day in data['prices']:
#             tt.append(day[0]/1000)
#             P.append(day[1])
#         df[asset+'_price'] = P
#         df['timestamp']  = tt

#     df['timestamp'] = tt

#     for asset in list(names.keys()):#[:LIMIT]:
        
#         diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(10)'] = change
        
#         diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
#         change = []
#         for i in range(10):
#             change.append('Nan')
#         for i in range(10,df.shape[0]):
#             change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
#         df[asset+'_%change(1)'] = change
        
#     for key in track.keys():
#         df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

#     df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
#     df  = df.iloc[::-1]
#     df  = df.reset_index()
#     df = df.drop(['index'], axis=1)

        
#     return df

In [6]:
# Coingecko Mapping for each token name to fetch historical prices
names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        "REN"   : "ren",
        "RENFIL": "renfil",
        "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }

In [7]:
# Funtion to give historical price changes for each token mentioned in Names Dictionary
def change():
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        P    = []
        if request.status_code != 200:
            data = {}
            data['prices'] = WBTC[ -LOOK_BACK: ]
            for day in data['prices']:
                P.append(day)
        else:   
            data = request.json()
            tt   = []
            

            if len(data['prices']) < LOOK_BACK:
                req = LOOK_BACK - len(data['prices'])
                addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
                data['prices'] = addition + data['prices']
                track[asset] = req
            else:
                data['prices'] = data['prices'][-LOOK_BACK:]

            for day in data['prices']:
                tt.append(day[0]/1000)
                P.append(day[1])
                
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    
    print("PRICES FETECHED")
    
    unknown_assets = ['PAX']
    for asset in list(names.keys()):#[:LIMIT]:
        try:
            change = []
            diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
            
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(10)'] = change

            diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
            change = []
            for i in range(10):
                change.append('Nan')
            for i in range(10,df.shape[0]):
                change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
            df[asset+'_%change(1)'] = change  
            
        except Exception as e:
            unknown_assets.append(asset)
            print("Error ", asset, e)
    
    for asset in unknown_assets:
        df[asset+'_%change(1)']  = df['WBTC_%change(1)'] 
        df[asset+'_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(10)'] = df['WBTC_%change(10)'] 
    df['PAX_%change(1)'] = df['WBTC_%change(1)'] 
        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df
change_df = change()

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'
CHECK POINT 


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

In [9]:
# query = '''
#                 {
#                 reserves
#                 (where:{
#                     symbol:"WBTC"
#                     })
#                 {
#                     id
#                     symbol
#                     name
#                     usageAsCollateralEnabled
#                         borrowingEnabled
#                     baseLTVasCollateral
#                     totalDeposits
#                     totalLiquidity
#                     totalATokenSupply
#                     reserveFactor
                    
#                 }
#                 }
#             '''

In [10]:
# V2_sample_transport       = RequestsHTTPTransport(
#         url='https://api.thegraph.com/subgraphs/name/aave/protocol-v2',
#         verify=True,
#         retries=5)

In [11]:
# Required Date
# By default set to current date
# requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate     = str((datetime.now() - timedelta(1)).date().strftime("%m-%d-%Y"))
requiredDate = '12-15-2021' #"01-10-2022" #mm-dd-yyyy

In [12]:
def VARDate(_date):
    '''
        Gets the VAR results, AaveV2 for a particular position and price change dataframe
        
        Args::
            _date(str): Date in "%m-%d-%Y" format
        
        Returns::
            aaveV2Res(pandas dataframe): AaveV2 Historical Data for the given date
            df (pandas dataframe): Price change dataframe
    '''
    # Aave V2 Data for particular date
    aaveV2Res = aaveV2HistoricalData(_date)
    
    # Price Change Dataframe
    df = change()
    
    # Flipping the date format
    reqDate = datetime.strptime(_date, '%m-%d-%Y').strftime('%Y-%m-%d')
    
    # Converting str to datetime
    df['date']= pd.to_datetime(df['date'])
    # Remove dates above the required date
    mask = (df['date'] <= reqDate)
    df = df.loc[mask]
    
    # Converting the date column back to str
    df['date']=df['date'].astype(str)
    
    REQUIRED_1 = []
    REQUIRED_10 = []
    historicalVAR = pd.DataFrame(columns=['date', 'VAR_10', 'VAR_1'])
    hist_ind = 0
    for required_row in df.iterrows():
        required_row = required_row[1]
        SUM_10 = []
        SUM_1 = []
            
        for row in aaveV2Res.iterrows():
            try:
                SUM_10.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(10)'])/100))
                SUM_1.append((row[1]['Position($,Billion)'] * float(required_row[ row[1]['Token'] +'_%change(1)'])/100))
            except Exception as e:
                print("ERROR", e)
                pass

        REQUIRED_1.append(sum(SUM_1))
        REQUIRED_10.append(sum(SUM_10))

        historicalVAR.loc[hist_ind] = [required_row['date']] + [sum(SUM_10)] + [sum(SUM_1)]
        hist_ind += 1 
        
    # Historical VAR for past 362 days    
    historicalVAR = historicalVAR[:365]
    
    
    
    REQUIRED_10 = historicalVAR.sort_values('VAR_10').iloc[3]
    REQUIRED_1 = historicalVAR.sort_values('VAR_1').iloc[3]

    REQUIRED_10_95 = historicalVAR.sort_values('VAR_10').iloc[9]
    REQUIRED_1_95 = historicalVAR.sort_values('VAR_1').iloc[9]

    var_date_10_99p = REQUIRED_10['date']
    var_date_10_95p = REQUIRED_10_95['date']
    var_date_1_99p = REQUIRED_1['date']
    var_date_1_95p = REQUIRED_1_95['date']
    
#     print(historicalVAR)
    
    testdf = pd.DataFrame(columns=['SYMBOL', 'totalLiquidity', 'totalDebt'])
    ind = 0
    
    AAVE_API_RESULT  = requests.get('https://aave-api-v2.aave.com/data/liquidity/v2?poolId=0xb53c1a33016b2dc2ff3653530bff1848a515c8c5&date='+_date).json()
    
    for i in AAVE_API_RESULT:
        testdf.loc[ind] = [i['symbol']] + [float(i['totalLiquidity']) * float(i['referenceItem']['priceInUsd']) ]\
                    + [float(i['totalDebt']) * float(i['referenceItem']['priceInUsd'] )] #+ [i['referenceItem']['priceInUsd'] * ]
        ind += 1

    TOTAL_LIABILITIES = aaveV2Res['totalLiquidity'].sum()
    TOTAL_ASSETS      = aaveV2Res['Assets'].sum()
    VAR_10_DAY_99_P = str(REQUIRED_10['VAR_10'])
    VAR_DATE_10_DAY_99_P = var_date_10_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['10_day_95p'] = str(REQUIRED_10_95['VAR_10'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_10_day_95p'] = var_date_10_95p

#     # transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1[5])
#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_99p'] = str(REQUIRED_1['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_99p'] = var_date_1_99p

#     transactionResponse["result"]["AAVEV2"]["data"]['1_day_95p'] = str(REQUIRED_1_95['VAR_1'])
#     transactionResponse["result"]["AAVEV2"]["data"]['var_date_1_day_95p'] = var_date_1_95p

    RELATIVE_VAR_ASSETS = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_ASSETS)
    RELATIVE_VAR_LIABILITIES = str(REQUIRED_10['VAR_10'] * pow(10,9) / TOTAL_LIABILITIES)
    #TOTAL_ASSETS
    #TOTAL_LIABILITIES
    return aaveV2Res, df, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P

In [13]:
# aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)
# Date in %m-%d-%Y format
aaveV2Res, priceChangeDf, TOTAL_ASSETS, TOTAL_LIABILITIES, VAR_10_DAY_99_P, VAR_DATE_10_DAY_99_P = VARDate(requiredDate)

PRICES FETECHED
Error  SUSD unsupported operand type(s) for -: 'list' and 'list'
Error  ENJ unsupported operand type(s) for -: 'list' and 'list'
Error  REN unsupported operand type(s) for -: 'list' and 'list'
Error  SNX unsupported operand type(s) for -: 'list' and 'list'


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


CHECK POINT 


In [14]:
# Checking aaveV2Res
aaveV2Res
# To be taken [Token, Position, Position($,Billion)]
# REMEBER TO PUT PLATFORM

,Token,CurrentPrice,availableLiquidity,decimals,totalLiquidity,utilizationRate,Assets,Position,"Position($,Billion)"
0,TUSD,0.993396,4.909780e+07,18.0,1.152567e+08,0.621094,6.648317e+07,-4.877355e+07,-0.048774
1,RAI,2.997620,2.068256e+06,18.0,3.107426e+07,0.552663,2.487442e+07,-6.199845e+06,-0.006200
2,GUSD,1.000000,2.911591e+06,2.0,1.439694e+07,0.760711,1.148535e+07,-2.911591e+06,-0.002912
3,YFI,20931.580402,1.469237e+03,18.0,3.294598e+07,0.042945,2.192518e+06,-3.075346e+07,-0.030753
4,BAT,1.002798,4.199938e+06,18.0,4.614137e+06,0.083232,4.024483e+05,-4.211688e+06,-0.004212
5,MANA,3.308269,1.337645e+07,18.0,4.566450e+07,0.024480,1.411599e+06,-4.425290e+07,-0.044253
6,DPI,231.478420,1.789095e+05,18.0,4.163930e+07,0.005984,2.256039e+05,-4.141369e+07,-0.041414
7,UNI,15.449653,2.650525e+06,18.0,4.138014e+07,0.116723,4.304442e+05,-4.094969e+07,-0.040950
8,WBTC,48723.711340,2.647879e+04,8.0,1.319985e+09,0.024686,2.984015e+07,-1.290145e+09,-1.290145
9,REN,0.477924,4.418143e+07,18.0,2.163059e+07,0.024041,5.152433e+05,-2.111534e+07,-0.021115


In [15]:
# Change in the price data for those particular dates

In [16]:
VAR_10_DAY_99_P

'-4.483380870529752'

In [17]:
VAR_DATE_10_DAY_99_P

'2021-01-06'

In [18]:
type(priceChangeDf['date'])

pandas.core.series.Series

In [19]:
# Token change in price for AAVE VAR Date
idx = priceChangeDf.index[priceChangeDf['date'] == VAR_DATE_10_DAY_99_P].tolist()
priceChangeVARDATE = priceChangeDf.loc[idx]
# _index = priceChangeDf.index[mask]
# priceChangeDf[_index]

In [20]:
priceChangeVARDATE["TUSD"+"_price"]

373    1.003462
Name: TUSD_price, dtype: float64

In [21]:
priceChangeVARDATE["TUSD"+"_%change(10)"]

373    0.170563
Name: TUSD_%change(10), dtype: object

In [22]:
# Contribution to VAR
# -1 * (TOKEN_PRICE * TOKEN_POSITION * TOKEN%CHANGE)

In [23]:
# PLATFORM, TOKEN, POSITION_ON_INPUT_DATE, POSITION_in_B_ON_INPUT_DATE, TOKEN_%_CHANGE_ON_VAR_DATE, CONTRIBTION TO VAR

In [24]:
column_names = ["PLATFORM", "TOKEN", "POSITION_ON_INPUT_DATE", "POSITION_ON_INPUT_DATE_BILLIONS",
               "TOKEN_%_CHANGE_ON_VAR_DATE", "CONTRIBUTION_TO_VAR"]
resultDf = pd.DataFrame(columns = column_names)

In [25]:
# Itearing through the position in Aave

# for token in aaveV2Res["Token"]:
for idx, row in aaveV2Res.iterrows():
    try:
        temp = []
        
        temp.append("AAVE")
        temp.append(str(aaveV2Res["Token"][idx]))
        temp.append(aaveV2Res["Position"][idx])
        temp.append(aaveV2Res["Position($,Billion)"][idx])
        temp.append(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])
        temp.append(float(-1 * float(aaveV2Res["Position($,Billion)"][idx]) * float(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])))
        
        
        
        # Add the values to the latest row
        df_length = len(resultDf)
        resultDf.loc[df_length] = temp
        
    except:
        print("Exception occured for the token: "+str(token))

# print(temp)

In [26]:
# Sample Final Res
resultDf

,PLATFORM,TOKEN,POSITION_ON_INPUT_DATE,POSITION_ON_INPUT_DATE_BILLIONS,TOKEN_%_CHANGE_ON_VAR_DATE,CONTRIBUTION_TO_VAR
0,AAVE,TUSD,-4.877355e+07,-0.048774,0.170563,0.008319
1,AAVE,RAI,-6.199845e+06,-0.006200,28.852430,0.178881
2,AAVE,GUSD,-2.911591e+06,-0.002912,0.293206,0.000854
3,AAVE,YFI,-3.075346e+07,-0.030753,5.766264,0.177333
4,AAVE,BAT,-4.211688e+06,-0.004212,8.584189,0.036154
5,AAVE,MANA,-4.425290e+07,-0.044253,3.946910,0.174662
6,AAVE,DPI,-4.141369e+07,-0.041414,53.720534,2.224766
7,AAVE,UNI,-4.094969e+07,-0.040950,85.284473,3.492373
8,AAVE,WBTC,-1.290145e+09,-1.290145,28.852430,37.223813
9,AAVE,REN,-2.111534e+07,-0.021115,28.852430,0.609229


### Compound Code

In [27]:
def change():

    names = {"DAI"  : 'dai',
        "GUSD"  : 'gemini-dollar',
        # "SUSD"  : 'susd',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDP"  : 'usdp',
        "USDT"  : "tether",
        "BAL"   : "balancer",
        "WETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "RAI"   : "rai",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "XSUSHI": "xsushi",
        "YFI"   : "yearn-finance",
        "BUSD"  : "binance-usd",
        "FEI"   : "fei-usd",
        "FRAX"  : "frax",
        "AAVE"  : "aave",
        "AMPL"  : "ampleforth",
        "BAT"   : "basic-attention-token",
        "CRV"   : "curve-dao-token",
        "DPI"   : "defipulse-index",
        # "ENJ"   : "enjin-coin",
        "KNC"   : "kyber-network-crystal",
        "MANA"  : "decentraland",
        # "REN"   : "ren",
        "RENFIL": "renfil",
        # "SNX"   :"synthetix-network-token",
        "ZRX"   : "0x"
        }
    columns = []
    columns.append('timestamp')
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]
        
        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
        df['timestamp']  = tt
    df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:
        
        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change
        
        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)

    df = df[:-10]

    print("CHECK POINT ")
    return df

In [28]:
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi",  
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

def change():    
    columns = []
    columns.append('timestamp')
    print(names)
    for key in list(names.keys()):
        columns.append(key+'_price')
        columns.append(key+'_%change(10)')
    df        = pd.DataFrame(columns = columns)
    LOOK_BACK = 375 * 2
    #LIMIT     = 5
    WBTC      = []
    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names['WBTC'])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    data      = request.json()
    WBTC      = data['prices']
    tt        = []
    P         = []
    for day in data['prices'][-LOOK_BACK:]:
        tt.append(day[0]/1000)
        P.append(day[1])
    df['WBTC_price'] = P
    df['timestamp']  = tt
    track = {}
    for asset in list(names.keys()):#[:LIMIT]:
        if asset == 'WBTC':
            continue
        print("DEBUG 1 ", asset)
        request = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[asset])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        data = request.json()
        tt = []
        P = []
        # print("DEBUG 1 ", data)
        if len(data['prices']) < LOOK_BACK:
            req = LOOK_BACK - len(data['prices'])
            addition = WBTC[-LOOK_BACK:  -LOOK_BACK + req ]
            data['prices'] = addition + data['prices']
            track[asset] = req
        else:
            data['prices'] = data['prices'][-LOOK_BACK:]

        for day in data['prices']:
            tt.append(day[0]/1000)
            P.append(day[1])
        df[asset+'_price'] = P
#         df['timestamp']  = tt
#     df['timestamp'] = tt
    for asset in list(names.keys()):#[:LIMIT]:

        diff_10 = pd.DataFrame(df[asset+'_price'].diff(periods=10) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_10.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(10)'] = change

        diff_1 = pd.DataFrame(df[asset+'_price'].diff(periods=1) )
        change = []
        for i in range(10):
            change.append('Nan')
        for i in range(10,df.shape[0]):
            change.append((diff_1.iloc[i][asset+'_price'] / df.iloc[i-10][asset+'_price']) * 100)
        df[asset+'_%change(1)'] = change        
    for key in track.keys():
        df[key+'_%change(10)'][:track[key]] = df['WBTC_%change(10)'][:track[key]]

    df['date'] = df['timestamp'].apply(lambda x : str(datetime.fromtimestamp(x).date()))
    df  = df.iloc[::-1]
    df  = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df[:-10]
    print("CHECK POINT ")
    return df[1:]

change_df = change()

# min_var = 9999999

{'DAI': 'dai', 'TUSD': 'true-usd', 'USDC': 'usd-coin', 'USDT': 'tether', 'ETH': 'ethereum', 'LINK': 'chainlink', 'MKR': 'maker', 'UNI': 'uniswap', 'WBTC': 'wrapped-bitcoin', 'WBTC2': 'wrapped-bitcoin', 'SUSHI': 'sushi', 'YFI': 'yearn-finance', 'AAVE': 'aave', 'BAT': 'basic-attention-token', 'ZRX': '0x', 'REP': 'augur', 'SAI': 'sai'}
DEBUG 1  DAI
DEBUG 1  TUSD
DEBUG 1  USDC
DEBUG 1  USDT
DEBUG 1  ETH
DEBUG 1  LINK
DEBUG 1  MKR
DEBUG 1  UNI
DEBUG 1  WBTC2
DEBUG 1  SUSHI
DEBUG 1  YFI
DEBUG 1  AAVE
DEBUG 1  BAT
DEBUG 1  ZRX
DEBUG 1  REP
DEBUG 1  SAI
CHECK POINT 


/home/nishgaba/anaconda3/envs/defi/lib/python3.6/site-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
change_df.head()

,timestamp,DAI_price,DAI_%change(10),TUSD_price,TUSD_%change(10),USDC_price,USDC_%change(10),USDT_price,USDT_%change(10),ETH_price,...,WBTC_%change(1),WBTC2_%change(1),SUSHI_%change(1),YFI_%change(1),AAVE_%change(1),BAT_%change(1),ZRX_%change(1),REP_%change(1),SAI_%change(1),date
1,1.642032e+09,1.001430,-0.0298551,1.001834,0.10263,1.001409,0.108038,1.000972,0.0488767,3378.035895,...,2.41734,2.41734,1.72256,2.63012,1.98119,4.5334,3.92047,3.23535,0,2022-01-13
2,1.641946e+09,0.999613,-0.494773,1.000153,-0.557577,0.999859,-0.508185,1.000418,-0.517607,3246.413723,...,2.03631,2.03631,4.23861,1.99777,5.13704,4.90393,3.49427,5.37757,0,2022-01-12
3,1.641859e+09,1.001071,0.170157,1.002476,0.424748,1.000715,0.0284644,1.001096,0.251778,3086.729002,...,-0.181559,-0.181559,-3.77384,-1.97316,-3.98116,-2.77533,-4.79252,-4.2775,0,2022-01-11
4,1.641773e+09,1.002857,-0.059015,1.000299,-0.169955,1.000593,-0.263729,0.999476,-0.185822,3156.967262,...,0.445028,0.445028,3.44492,-1.71643,-1.17769,0.412506,1.95901,0.724551,0,2022-01-10
5,1.641686e+09,0.997979,-0.511445,0.997312,-0.101253,0.999403,-0.581565,0.997693,-0.457192,3101.040902,...,-0.653672,-0.653672,-6.24161,-8.69001,0.953256,-4.65892,-5.25208,-4.46708,0,2022-01-09


In [30]:
# !mkdir AAVE_COMPOUND_DATA
!mkdir COMPOUND_DATA

mkdir: cannot create directory ‘COMPOUND_DATA’: File exists


In [31]:
# names = {"DAI"  : 'dai',
#         "GUSD"  : 'gemini-dollar',
#         # "SUSD"  : 'susd',
#         "TUSD"  : 'true-usd',
#         "USDC"  : 'usd-coin',
#         # "USDP"  : 'usdp',
#         "USDT"  : "tether",
#         "BAL"   : "balancer",
#         "WETH"   : "ethereum",
#         "LINK"  : "chainlink",
#         "MKR"   : "maker",
#         "RAI"   : "rai",
#         "UNI"   : "uniswap",
#         "WBTC"  : "wrapped-bitcoin",
#         "XSUSHI": "xsushi",
#         "YFI"   : "yearn-finance",
#         "BUSD"  : "binance-usd",
#         "FEI"   : "fei-usd",
#         "FRAX"  : "frax",
#         "AAVE"  : "aave",
#         "AMPL"  : "ampleforth",
#         "BAT"   : "basic-attention-token",
#         "CRV"   : "curve-dao-token",
#         "DPI"   : "defipulse-index",
#         # # "ENJ"   : "enjin-coin",
#         "KNC"   : "kyber-network-crystal",
#         "MANA"  : "decentraland",
#         # "REN"   : "ren",
#         "RENFIL": "renfil",
#         # "SNX"   :"synthetix-network-token",
#         "ZRX"   : "0x"
#         }

In [32]:
# requiredDate

In [33]:
# Compound Tokens
names = {"DAI"  : 'dai',
        "TUSD"  : 'true-usd',
        "USDC"  : 'usd-coin',
        "USDT"  : "tether",
        "ETH"   : "ethereum",
        "LINK"  : "chainlink",
        "MKR"   : "maker",
        "UNI"   : "uniswap",
        "WBTC"  : "wrapped-bitcoin",
        "WBTC2"  : "wrapped-bitcoin",
        "SUSHI" :"sushi", # check -- - - - - -- 
        "YFI"   : "yearn-finance",
        "AAVE"  : "aave",
        "BAT"   : "basic-attention-token",
        "ZRX"   : "0x",
        "REP"   : "augur",
        "SAI"   : "sai",
        }

In [34]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
for ctoken in res.json()['cToken'][3:]:
    
    print(ctoken['symbol'])
    if ctoken['symbol'] == 'cCOMP':
        continue

    ind += 1
    for key in ctoken.keys():
        try:
            df.at[ind,key ] = ctoken[key]['value']
        except:
            df.at[ind,key ] = ctoken[key]

    

    address   = ctoken['token_address']

    request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
    prices_df = pd.DataFrame(columns=['timestamp', 'price'])
    timestamp = []
    prices    = []

    for req in request.json()['prices']:
        timestamp.append(int(float(req[0])/1000))
        prices.append(req[1])

    prices_df['timestamp'] = timestamp
    prices_df['price']     = prices

    date= []
    for time in timestamp:
        date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
    prices_df['date']  = date

cWBTC
cMKR
cZRX
cYFI
cSUSHI
cLINK
cBAT
cTUSD
cCOMP
cAAVE
cUNI
cWBTC2
cUSDT
cUSDC
cETH
cDAI


In [62]:
prices_df

,timestamp,price,date
0,1574121600,1.000651,2019-11-19
1,1574208000,0.995411,2019-11-20
2,1574294400,0.991591,2019-11-21
3,1574380800,0.991102,2019-11-22
4,1574467200,0.992850,2019-11-23
...,...,...,...
783,1641772800,1.002857,2022-01-10
784,1641859200,1.001071,2022-01-11
785,1641945600,0.999613,2022-01-12
786,1642032000,1.001430,2022-01-13


In [63]:
requiredDate
datetime.strptime(requiredDate, '%m-%d-%Y').strftime('%Y-%m-%d')

'2021-12-15'

In [64]:
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
list(res.json()['cToken'][0])

['borrow_cap',
 'borrow_rate',
 'cash',
 'collateral_factor',
 'comp_borrow_apy',
 'comp_supply_apy',
 'exchange_rate',
 'interest_rate_model_address',
 'name',
 'number_of_borrowers',
 'number_of_suppliers',
 'reserve_factor',
 'reserves',
 'supply_rate',
 'symbol',
 'token_address',
 'total_borrows',
 'total_supply',
 'underlying_address',
 'underlying_name',
 'underlying_price',
 'underlying_symbol']

In [65]:
ind  = 0 
res  = requests.get("https://api.compound.finance/api/v2/ctoken")
df   = pd.DataFrame(columns = list(res.json()['cToken'][0].keys()))
# MASTER_DATE = '2021-12-20'
MASTER_DATE = datetime.strptime(requiredDate, '%m-%d-%Y').strftime('%Y-%m-%d')
for ctoken in res.json()['cToken']:
    try:
        ind += 1
        for key in ctoken.keys():
            try:
                df.at[ind,key ] = ctoken[key]['value']
            except:
                df.at[ind,key ] = ctoken[key]


        address   = ctoken['token_address']

        request   = requests.get("https://api.coingecko.com/api/v3/coins/"+ str(names[ctoken['symbol'][1:]])+ "/market_chart?vs_currency=usd&days=max&interval=daily")
        prices_df = pd.DataFrame(columns=['timestamp', 'price'])
        timestamp = []
        prices    = []

        for req in request.json()['prices']:
            timestamp.append(int(float(req[0])/1000))
            prices.append(req[1])

        prices_df['timestamp'] = timestamp
        prices_df['price']     = prices

        date= []
        for time in timestamp:
            date.append(datetime.fromtimestamp(time).strftime('%Y-%m-%d'))
        prices_df['date']  = date

#         datestamp = '2021-12-14'
        datestamp = MASTER_DATE
        max_timstamp = int(datetime.strptime(datestamp, '%Y-%m-%d').timestamp()) + 86400
        SUPPLY_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        BORROW_DF = pd.DataFrame(columns=['block_number', 'block_timestamp','value'])
        DF = pd.DataFrame()
        BIND = 0
        SIND = 0
        for _ in range(5):
            min_timestamp = max_timstamp  -  30 * 86400 
            res = requests.get("https://api.compound.finance/api/v2/market_history/graph?asset=%s&min_block_timestamp=%s&max_block_timestamp=%s&num_buckets=50" % ( address,min_timestamp, max_timstamp));    
            max_timstamp = max_timstamp   - 30 * 86400 
            temp = res.json()


            for tem in temp['total_borrows_history']:
                BORROW_DF.loc[BIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]                     
                BIND += 1


            for tem in temp['total_supply_history']:
                SUPPLY_DF.loc[SIND] = [tem['block_number']] + [tem['block_timestamp']] + [tem['total']['value']]
                SIND += 1                      


            print("***")

        BORROW_DF = BORROW_DF.sort_values(['block_timestamp'])
        BORROW_DF['DATE'] = BORROW_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        BORROW_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)
        SUPPLY_DF = SUPPLY_DF.sort_values(['block_timestamp'])
        SUPPLY_DF['DATE'] = SUPPLY_DF['block_timestamp'].apply(lambda x: datetime.fromtimestamp(x).date())
        SUPPLY_DF.drop_duplicates(subset = ['DATE'], keep = 'first', inplace = True)

        SUPPLY_DF['supply_tokens'] = pd.to_numeric(SUPPLY_DF['value']) *  float(temp['exchange_rates'][-1]['rate'])


        BORROW_DF = BORROW_DF[::-1]
        SUPPLY_DF = SUPPLY_DF[::-1]
        DF['timestamp']    = BORROW_DF['block_timestamp']
        DF['BORROW']       = BORROW_DF['value']
        DF['SUPPLY']       = SUPPLY_DF['supply_tokens']
        DF['DATE']         = BORROW_DF['DATE']
        DF['SUPPLY_VALUE'] = SUPPLY_DF['value']


        assigned_price = []
        for indW in DF['DATE'] :
            a = prices_df[prices_df['date']==str(indW)]
            assigned_price.append(a['price'].values[0])

        DF['prices'] = assigned_price

        DF['total_supply'] =  DF['SUPPLY'] * DF['prices']
        DF['total_borrow'] =  pd.to_numeric(DF['BORROW']) * pd.to_numeric(DF['prices'])
        DF['POSITION']     =  DF['total_borrow'] - DF['total_supply']
        DF['POSITION($)']  =  DF['POSITION'] / pow(10,9)
        DF.to_csv('COMPOUND_DATA/DF_'+ctoken['symbol'][1:] + '.csv')



        print("**********************", DF.shape)
    except Exception as e:
        print("ERROR ", e, ctoken['symbol'])
    

***
***
***
***
***
********************** (100, 10)
***
***
***
***
***
ERROR  index 0 is out of bounds for axis 0 with size 0 cSAI
ERROR  'USDP' cUSDP
***
***
***
***
***
********************** (142, 10)
***
***
***
***
***
********************** (130, 10)
***
***
***
***
***
********************** (148, 10)
***
***
***
***
***
********************** (130, 10)
***
***
***
***
***
********************** (142, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (142, 10)
ERROR  'COMP' cCOMP
***
***
***
***
***
********************** (143, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)
***
***
***
***
***
********************** (150, 10)


In [66]:
fils = os.listdir('COMPOUND_DATA/')

In [67]:
cols = []
for fil in fils:
    try:
        aa = fil.split('.')[0].split('_')
    #     print(aa[1])
        cols.append(aa[1])
    except:
        pass

In [68]:
FINAL ={}
for key in cols:
    try:
        df = pd.read_csv('COMPOUND_DATA/'+'DF_'+key+'.csv')
        df = df.sort_values(['timestamp'])
#         print(df.head())
        df = df[::-1]
        FINAL[key] = {'Assets': df['POSITION($)'],
                      'date':df['DATE'],
                      'MARKET':df['total_supply'],
                      'BORROW':df['total_borrow']
                     }
    
    except Exception as e:
        print(e, key)


In [69]:
ff = []
for f in FINAL['ETH']['date']:
    ff.append(f)

In [70]:
FINAL["AAVE"]["date"]

0      2021-12-13
1      2021-12-12
2      2021-12-11
3      2021-12-10
4      2021-12-09
          ...    
138    2021-07-26
139    2021-07-25
140    2021-07-24
141    2021-07-23
142    2021-07-22
Name: DATE, Length: 143, dtype: object

In [71]:
FINAL_DF = pd.DataFrame(columns= cols)
ind = 0
for i in range(100):
    assets = []
    for col in cols:
        try:
            assets.append(FINAL[col]['Assets'][i])
        except:
            assets.append([])
    
    FINAL_DF.loc[ind] = assets
    ind += 1

In [72]:
FINAL_DF['DATE'] = ff[:100]
FINAL_DF['timestamp'] = FINAL_DF['DATE'].apply(lambda x: pd.Timestamp(x).timestamp())
FINAL_DF = FINAL_DF.sort_values('timestamp')
FINAL_DF = FINAL_DF[::-1]

In [73]:
# Assets for Compound Datewise
FINAL_DF

,AAVE,BAT,DAI,ETH,LINK,MKR,REP,SUSHI,TUSD,UNI,USDC,USDT,WBTC,WBTC2,YFI,ZRX,DATE,timestamp
0,-0.002040,-0.114200,-0.846538,-5.580043,-0.036416,-0.002633,-0.000010,-0.000554,-0.038614,-0.120831,-1.074861,-0.277186,-0.121122,-1.528197,-0.000731,-0.091199,2021-12-15,1.639526e+09
1,-0.002273,-0.110935,-0.872696,-5.483535,-0.035025,-0.001273,0.000007,-0.000478,-0.038740,-0.107994,-1.083012,-0.267171,-0.116214,-1.483477,-0.000803,-0.089369,2021-12-14,1.639440e+09
2,-0.002132,-0.129578,-0.869851,-5.993492,-0.040418,-0.001367,0.000007,-0.000488,-0.038821,-0.122993,-1.082755,-0.249741,-0.117592,-1.583484,-0.000783,-0.099822,2021-12-13,1.639354e+09
3,-0.002250,-0.123206,-0.872525,-5.954120,-0.039332,-0.001233,0.000007,-0.000385,-0.038594,-0.115113,-1.053319,-0.237907,-0.124414,-1.560761,-0.000727,-0.100829,2021-12-12,1.639267e+09
4,-0.002538,-0.117118,-0.877906,-5.714411,-0.036542,-0.001181,0.000008,-0.000419,-0.038629,-0.107881,-1.020039,-0.203815,-0.124483,-1.489819,-0.000762,-0.095976,2021-12-11,1.639181e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,-0.011622,-0.085293,-1.121241,-5.596392,-0.146552,-0.000464,-0.000066,-0.000539,-0.025585,-0.223366,-1.071292,-0.169048,-0.137888,-1.678250,-0.000246,-0.107232,2021-09-11,1.631318e+09
96,-0.013656,-0.091589,-1.154804,-6.110581,-0.156354,-0.000409,-0.000041,-0.000675,-0.025659,-0.247911,-0.936270,-0.121527,-0.136197,-1.539744,-0.000231,-0.108311,2021-09-10,1.631232e+09
97,-0.013851,-0.086336,-1.147865,-6.207497,-0.155891,-0.000441,-0.000041,-0.000705,-0.025582,-0.243406,-1.013690,-0.126775,-0.134452,-1.517037,-0.000153,-0.136693,2021-09-09,1.631146e+09
98,-0.013372,-0.088002,-1.144708,-6.111803,-0.162653,-0.000457,-0.000041,-0.000468,-0.025613,-0.255859,-1.019216,-0.121855,-0.130973,-1.538517,-0.000013,-0.138833,2021-09-08,1.631059e+09


In [96]:
change_df

,timestamp,DAI_price,DAI_%change(10),TUSD_price,TUSD_%change(10),USDC_price,USDC_%change(10),USDT_price,USDT_%change(10),ETH_price,...,WBTC_%change(1),WBTC2_%change(1),SUSHI_%change(1),YFI_%change(1),AAVE_%change(1),BAT_%change(1),ZRX_%change(1),REP_%change(1),SAI_%change(1),date
1,1.642032e+09,1.001430,-0.0298551,1.001834,0.10263,1.001409,0.108038,1.000972,0.0488767,3378.035895,...,2.41734,2.41734,1.72256,2.63012,1.98119,4.5334,3.92047,3.23535,0,2022-01-13
2,1.641946e+09,0.999613,-0.494773,1.000153,-0.557577,0.999859,-0.508185,1.000418,-0.517607,3246.413723,...,2.03631,2.03631,4.23861,1.99777,5.13704,4.90393,3.49427,5.37757,0,2022-01-12
3,1.641859e+09,1.001071,0.170157,1.002476,0.424748,1.000715,0.0284644,1.001096,0.251778,3086.729002,...,-0.181559,-0.181559,-3.77384,-1.97316,-3.98116,-2.77533,-4.79252,-4.2775,0,2022-01-11
4,1.641773e+09,1.002857,-0.059015,1.000299,-0.169955,1.000593,-0.263729,0.999476,-0.185822,3156.967262,...,0.445028,0.445028,3.44492,-1.71643,-1.17769,0.412506,1.95901,0.724551,0,2022-01-10
5,1.641686e+09,0.997979,-0.511445,0.997312,-0.101253,0.999403,-0.581565,0.997693,-0.457192,3101.040902,...,-0.653672,-0.653672,-6.24161,-8.69001,0.953256,-4.65892,-5.25208,-4.46708,0,2022-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,1.578614e+09,1.002732,2.12226,1.000020,-0.257931,1.000419,-0.0833051,0.999479,0.120977,137.860559,...,-3.10762,-3.10762,-3.10762,-3.10762,-3.10762,1.16746,-2.43441,-2.62453,0.199728,2020-01-10
736,1.578528e+09,1.000406,1.61287,0.996705,-0.35493,0.996342,-0.389219,0.996537,-0.319909,140.273935,...,-0.667996,-0.667996,-0.667996,-0.667996,-0.667996,-5.40615,12.0742,-0.331698,0.113107,2020-01-09
737,1.578442e+09,0.995636,2.42278,0.999350,-0.317283,0.999140,-0.249331,0.997681,-0.16141,143.019432,...,4.64645,4.64645,4.64645,4.64645,4.64645,0.925568,-3.80894,-1.29527,-0.450304,2020-01-08
738,1.578355e+09,0.998680,2.48368,0.998690,-0.22173,0.999543,-0.198063,0.998183,-0.167694,143.806398,...,5.94334,5.94334,5.94334,5.94334,5.94334,2.30919,4.73891,3.83729,0.142036,2020-01-07


In [75]:
VAR = []
VAR_DATE = []
for ind, ROW in enumerate(FINAL_DF.iterrows()):
    REQUIRED_10 = []
    for required_row in change_df[ind:ind+365].iterrows():
        SUM_10 = []
        for key in ROW[1].keys()[:-3]:
            if key == 'WBTC2':
                key = 'WBTC'
            if isinstance(ROW[1][key], list):
                ROW[1][key] = 0
            SUM_10.append((float(ROW[1][key]) * required_row[1][key+'_%change(10)'])/100)
        REQUIRED_10.append(sum(SUM_10))
    VAR.append(sorted(REQUIRED_10, key=float)[3])
    VAR_DATE.append(change_df[ind:ind+365].at[REQUIRED_10.index(sorted(REQUIRED_10, key=float)[3]) + ind, 'date'])

In [76]:
BORROW_DF

,block_number,block_timestamp,value,DATE
49,13804249,1639541156,53164.487548679693788557,2021-12-15
47,13796365,1639437476,53435.396107370703671632,2021-12-14
46,13792423,1639385636,53409.289204436265006701,2021-12-13
44,13784538,1639281956,51625.151603059459040437,2021-12-12
42,13776654,1639178277,51376.616714425814291543,2021-12-11
...,...,...,...,...
207,12850235,1626995879,124555.306205228512046093,2021-07-23
206,12846293,1626944039,128746.836115987861175034,2021-07-22
204,12838408,1626840359,130843.267114626298625068,2021-07-21
202,12830524,1626736679,126643.850730796175745663,2021-07-20


In [77]:
# VAR_DATE

In [78]:
FINAL_DF["DATE"]

0     2021-12-15
1     2021-12-14
2     2021-12-13
3     2021-12-12
4     2021-12-11
         ...    
95    2021-09-11
96    2021-09-10
97    2021-09-09
98    2021-09-08
99    2021-09-07
Name: DATE, Length: 100, dtype: object

In [79]:
# Sample Final Res Columns
resultDf.columns

Index(['PLATFORM', 'TOKEN', 'POSITION_ON_INPUT_DATE',
       'POSITION_ON_INPUT_DATE_BILLIONS', 'TOKEN_%_CHANGE_ON_VAR_DATE',
       'CONTRIBUTION_TO_VAR'],
      dtype='object')

In [80]:
# resultDf

In [81]:
FINAL_DF.columns[:-2][0]

'AAVE'

In [82]:
requiredDate

'12-15-2021'

In [93]:
FINAL_DF["AAVE"].loc[0]

-0.0020397422055186405

In [95]:
VAR_DATE[0]

'2021-05-11'

In [102]:
idx = change_df.index[change_df['date'] == str(VAR_DATE[0])]
priceChangeVARDATE = change_df.loc[idx]

In [103]:
priceChangeVARDATE

,timestamp,DAI_price,DAI_%change(10),TUSD_price,TUSD_%change(10),USDC_price,USDC_%change(10),USDT_price,USDT_%change(10),ETH_price,...,WBTC_%change(1),WBTC2_%change(1),SUSHI_%change(1),YFI_%change(1),AAVE_%change(1),BAT_%change(1),ZRX_%change(1),REP_%change(1),SAI_%change(1),date
248,1.620691e+09,1.003394,0.400227,1.001942,-0.0250118,0.999028,-0.159781,1.007187,0.58062,3979.608652,...,-3.97152,-3.97152,-11.7483,16.8803,-5.47237,-7.90729,-11.2555,-14.9401,4.52943,2021-05-11


In [106]:
priceChangeVARDATE[token+"_%change(10)"].iloc[0]

-2.571297458079215

In [108]:
idx = change_df.index[change_df['date'] == str(VAR_DATE[0])]
priceChangeVARDATE = change_df.loc[idx]
    
for token in list(FINAL_DF.columns[:-2]):
    try:
#         print(token)
        temp = []
        
        temp.append("COMPOUND")
        temp.append(str(token))
        temp.append(float(FINAL_DF[token].loc[0]*pow(10,9)))
        temp.append(float(FINAL_DF[token].loc[0]))
        temp.append(priceChangeVARDATE[token+"_%change(10)"].iloc[0])
        temp.append(float(-1 * float(FINAL_DF[token].loc[0]) * float(priceChangeVARDATE[token+"_%change(10)"].iloc[0])))
        
        # Add the values to the latest row
        df_length = len(resultDf)
        resultDf.loc[df_length] = temp
        
    except:
        print("Exception occured for the token: "+str(token))
# print(temp)

AAVE
BAT
DAI
ETH
LINK
MKR
REP
SUSHI
TUSD
UNI
USDC
USDT
WBTC
WBTC2
YFI
ZRX
['COMPOUND', 'ZRX', -91199366.10187517, -0.09119936610187517, -2.571297458079215, -0.23450069823618733]


In [112]:
resultDf

,PLATFORM,TOKEN,POSITION_ON_INPUT_DATE,POSITION_ON_INPUT_DATE_BILLIONS,TOKEN_%_CHANGE_ON_VAR_DATE,CONTRIBUTION_TO_VAR
0,AAVE,TUSD,-4.877355e+07,-0.048774,0.170563,0.008319
1,AAVE,RAI,-6.199845e+06,-0.006200,28.852430,0.178881
2,AAVE,GUSD,-2.911591e+06,-0.002912,0.293206,0.000854
3,AAVE,YFI,-3.075346e+07,-0.030753,5.766264,0.177333
4,AAVE,BAT,-4.211688e+06,-0.004212,8.584189,0.036154
5,AAVE,MANA,-4.425290e+07,-0.044253,3.946910,0.174662
6,AAVE,DPI,-4.141369e+07,-0.041414,53.720534,2.224766
7,AAVE,UNI,-4.094969e+07,-0.040950,85.284473,3.492373
8,AAVE,WBTC,-1.290145e+09,-1.290145,28.852430,37.223813
9,AAVE,REN,-2.111534e+07,-0.021115,28.852430,0.609229


In [111]:
resultDf.to_csv(str(requiredDate)+"_AAVE_COMPOUND_VAR.csv", index=False)

In [84]:

# # for token in aaveV2Res["Token"]:
# for idx, row in aaveV2Res.iterrows():
#     try:
#         temp = []
        
#         temp.append("AAVE")
#         temp.append(str(aaveV2Res["Token"][idx]))
#         temp.append(aaveV2Res["Position"][idx])
#         temp.append(aaveV2Res["Position($,Billion)"][idx])
#         temp.append(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])
#         temp.append(float(-1 * float(aaveV2Res["Position($,Billion)"][idx]) * float(priceChangeVARDATE[str(aaveV2Res["Token"][idx])+"_%change(10)"].iloc[0])))
        
        
        
#         # Add the values to the latest row
#         df_length = len(resultDf)
#         resultDf.loc[df_length] = temp
        
#     except:
#         print("Exception occured for the token: "+str(token))

# # print(temp)

In [90]:
# idx = FINAL_DF.index[FINAL_DF['DATE'] == MASTER_DATE].tolist
# print(idx)
# # COMPpriceChangeVARDATE = FINAL_DF.loc[idx]

<bound method IndexOpsMixin.tolist of Int64Index([0], dtype='int64')>


In [91]:
# FINAL_DF.index[FINAL_DF['DATE'] == MASTER_DATE]

Int64Index([0], dtype='int64')

In [87]:
# FINAL_DF["DATE"]

0     2021-12-15
1     2021-12-14
2     2021-12-13
3     2021-12-12
4     2021-12-11
         ...    
95    2021-09-11
96    2021-09-10
97    2021-09-09
98    2021-09-08
99    2021-09-07
Name: DATE, Length: 100, dtype: object

In [61]:
# MASTER_DATE

'2021-12-15'